In [ ]:
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
from vit_pytorch import ViT
import torch.optim as optim
import matplotlib.pyplot as plt
import timm

batch_size=32

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# 使用する訓練画像の入ったフォルダ(ルート)
TRAIN_PATH =  

# 学習用データと検証用データに分割する
k=5
#skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

# ImageFolderを使用してデータセットをロード
dataset = datasets.ImageFolder(root=TRAIN_PATH, transform=transform)

from torch.utils.data import random_split

# データセットの分割（訓練データとテストデータ）
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# データローダの作成
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('100','200','250')



In [2]:
net = timm.create_model('vit_large_patch32_224.orig_in21k', pretrained=True, num_classes=3)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = net.to(device)

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(),lr=0.001)

epochs = 300
# 訓練およびテストの損失と精度を記録するためのリストを初期化
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []
for epoch in range(0, epochs):
    epoch_train_loss = 0
    epoch_train_acc = 0
    epoch_test_loss = 0
    epoch_test_acc = 0
    model.train()
    for data in train_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()/len(train_loader)
        acc = (outputs.argmax(dim=1) == labels).float().mean()
        epoch_train_acc += acc/len(train_loader)
    model.eval()
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            epoch_test_loss += loss.item()/len(test_loader)
            test_acc = (outputs.argmax(dim=1) == labels).float().mean()
            epoch_test_acc += test_acc/len(test_loader)
    # エポックごとの損失と精度をリストに追加
    train_losses.append(epoch_train_loss)
    train_accuracies.append(epoch_train_acc.item())
    test_losses.append(epoch_test_loss)
    test_accuracies.append(epoch_test_acc.item())
    print(f'Epoch {epoch+1} : train acc. {epoch_train_acc:.2f} train loss {epoch_train_loss:.2f}')
    print(f'Epoch {epoch+1} : test acc. {epoch_test_acc:.2f} test loss {epoch_test_loss:.2f}')
# 学習経過のグラフをプロット
plt.figure(figsize=(12, 4))
# 損失のグラフ
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.yscale('log')
plt.title('Loss over epochs')
plt.legend()
# 精度のグラフ
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.yscale('log')
plt.title('Accuracy over epochs')
plt.legend()
plt.show()